In [26]:
# 사용할 모듈 불러오기 
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re
import xgboost

In [27]:
# 데이터 불러오기
items = pd.read_csv("./data/items.csv")
shops = pd.read_csv("./data/shops.csv")
train = pd.read_csv("./data/sales_train.csv")
test = pd.read_csv("./data/test.csv")
item_categories = pd.read_csv('./data/item_categories.csv')

In [28]:
#  date의 자료형을 object  -> dattime 으로 바꾸고 싶다. 
# 변환하는 과정  ->
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")
train["date"]. head(3) 

0   2013-01-02
1   2013-01-03
2   2013-01-05
Name: date, dtype: datetime64[ns]

In [29]:
train['item_price'] = ((train["item_price"] <= 50000) & (train["item_price"] > 0))
train['item_cnt_day'] = ((train["item_cnt_day"] <= 1000) & (train["item_cnt_day"] > 0))

In [30]:
pivot_for_train = train.pivot_table(index = ['date_block_num','shop_id','item_id'] , values=['item_cnt_day'],aggfunc='sum')
pivot_for_train.rename(columns = {'item_cnt_day':'item_cnt_month'}, inplace = True)
pivot_for_train

item_cnt_month
date_block_num shop_id item_id                
0              0       32                    4
                       33                    3
                       35                    1
                       43                    1
                       51                    2
...                                        ...
33             59      22087                 3
                       22088                 2
                       22091                 1
                       22100                 1
                       22102                 1

[1609124 rows x 1 columns]

In [31]:
train = pivot_for_train.reset_index()
train

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,4
1,0,0,33,3
2,0,0,35,1
3,0,0,43,1
4,0,0,51,2
...,...,...,...,...
1609119,33,59,22087,3
1609120,33,59,22088,2
1609121,33,59,22091,1
1609122,33,59,22100,1


In [32]:
x_train = train[train['date_block_num'] < 33]
x_train = x_train.drop('item_cnt_month', axis = 1)
y_train = train[train['date_block_num'] < 33]['item_cnt_month']

x_valid = train[train['date_block_num'] == 33]
x_valid = x_valid.drop('item_cnt_month', axis = 1)
y_valid = train[train['date_block_num'] == 33]['item_cnt_month']

x_test = train[train['date_block_num'] == 34]
x_test = x_test.drop('item_cnt_month', axis = 1)
y_test = train[train['date_block_num'] == 34]['item_cnt_month']

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [35]:
# max_attributes = x_valid.shape[1]
# depth_range = range(1, max_attributes)

for depth in range(1,20):
    rand_clf = RandomForestRegressor(n_estimators=200, max_depth = depth)
    model = rand_clf.fit(x_train, y_train)
    valid_acc = model.predict(x_valid)
    
    print("Current max depth: ", depth, "\n")
    print("Accuracy:", r2_score(y_valid, valid_acc))

Current max depth:  1 

Accuracy: -0.013960445249928455
Current max depth:  2 

Accuracy: -0.011230703313268586
Current max depth:  3 

Accuracy: 0.15321491900929174
Current max depth:  4 

Accuracy: 0.16963370292954005
Current max depth:  5 

Accuracy: 0.1768045853474347
Current max depth:  6 

Accuracy: 0.18850323881818232
Current max depth:  7 

Accuracy: 0.20729872737671795
Current max depth:  8 

Accuracy: 0.22213302930847645
Current max depth:  9 

Accuracy: 0.24032606236571008
Current max depth:  10 

Accuracy: 0.26926991917974863
Current max depth:  11 

Accuracy: 0.29779671593096335
Current max depth:  12 

Accuracy: 0.3252655365921643
Current max depth:  13 

Accuracy: 0.3457097482437069
Current max depth:  14 

Accuracy: 0.36811952294484296
Current max depth:  15 

Accuracy: 0.3834776283907696
Current max depth:  16 

Accuracy: 0.3967234356519822
Current max depth:  17 

Accuracy: 0.40282676254473504
Current max depth:  18 

Accuracy: 0.4055800996790401
Current max depth:  1